In [1]:
#Importing Libraries
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
from sklearn.model_selection import train_test_split

In [3]:
# Reading the Dataset
dataset = pd.read_csv('twitter_dataset.csv')
dataset

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0000,yes,1.0000,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0000,yes,1.0000,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0000,yes,1.0000,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13866,13867,No candidate mentioned,1.0000,yes,1.0000,Negative,0.7991,Abortion,0.6014,No candidate mentioned,...,yes,7,Negative,Abortion\r\nWomen's Issues (not abortion though),RT @cappy_yarbrough: Love to see men who will ...,NaN,2015-08-07 09:29:43 -0700,629690895479250944,Como,NaN
13867,13868,Mike Huckabee,0.9611,yes,1.0000,Positive,0.7302,None of the above,0.9229,Mike Huckabee,...,yes,1,NaN,NaN,RT @georgehenryw: Who thought Huckabee exceede...,NaN,2015-08-07 09:25:02 -0700,629689719056568320,USA,NaN
13868,13869,Ted Cruz,1.0000,yes,1.0000,Positive,0.8051,None of the above,0.9647,Ted Cruz,...,yes,67,Positive\r\nNeutral,NaN,"RT @Lrihendry: #TedCruz As President, I will a...",NaN,2015-08-07 07:19:18 -0700,629658075784282112,NaN,NaN
13869,13870,Donald Trump,1.0000,yes,1.0000,Negative,1.0000,Women's Issues (not abortion though),0.9202,Donald Trump,...,yes,149,NaN,Women's Issues (not abortion though),RT @JRehling: #GOPDebate Donald Trump says tha...,NaN,2015-08-07 09:54:04 -0700,629697023663546368,NaN,NaN


In [4]:
dataset = dataset[['text','sentiment']]
dataset

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


In [5]:
dataset = dataset[dataset['sentiment']!= 'Neutral']
dataset

,text,sentiment
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",Positive
6,RT @warriorwoman91: I liked her and was happy ...,Negative
...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


In [6]:
# Data Preprocessing
train,test = train_test_split(dataset,test_size=0.1)
train

,text,sentiment
4365,@FoxNews Meghan Kelly was awful last night. Sh...,Negative
4036,RT @HeatherChilders: 'God has blessed the Rep ...,Positive
2329,Tried watching the #GOPDebate for entertainmen...,Negative
1331,Look at that affirmative action letting that b...,Negative
11140,It seems the candidates used their 1st amendme...,Negative
...,...,...
11726,RT @RWSurferGirl: Why doesn't Chris Wallace as...,Negative
13577,RT @RWSurferGirl: Fox is cherry picking the ca...,Negative
495,RT @RedStateJake: I'd pay some serious $$$ to ...,Negative
4342,"RT @sarahkendzior: ""If I were trying to destro...",Negative


In [7]:
test

,text,sentiment
7041,You know it's bad when Trump comes out looking...,Negative
152,"RT @sallykohn: Hey look, it’s 10 men standing ...",Negative
1034,Not much fact checking of the #GOPDebate becau...,Negative
554,I had to laugh when #Huck mentioned the pimps....,Positive
5215,The GOP candidates were grilled. Hard. Expec...,Negative
...,...,...
10334,RT @SupermanHotMale: Dr. Carson talking about ...,Negative
13559,"RT @AvidHeather: If Walker, Paul or Huckabee h...",Negative
13587,"RT @SupermanHotMale: Translation, Jeb Bush, I ...",Negative
2296,#GOPDebate in a nutshell via Vote True Blue 20...,Negative


In [8]:
train['text'][9720]

'RT @RWSurferGirl: It is very disappointing that Fox News is not conducting a "Fair &amp; Balanced" Debate. 🇺🇸 #GOPDebate  #GOPDebates'

In [10]:
pattern = "(#\w+)|(RT\s@\w+:)|(http.*)|(@\w+)"

In [11]:
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [14]:
def Clean_text(data):
    tweets = []
    sentiments = []
    for index,row in data.iterrows():
        sentence = re.sub(pattern,'',row.text)
        words = [e.lower() for e in sentence.split()]
        words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords.words('english')]
        words = ' '.join(words)
        tweets.append(words)
        sentiments.append(row.sentiment)
    return tweets,sentiments

In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [20]:
train_tweets,train_sentiments = Clean_text(train)

In [22]:
final_data = {'tweets':train_tweets,'sentiments':train_sentiments}

In [24]:
processed_data = pd.DataFrame(final_data)

In [25]:
processed_data

,tweets,sentiments
0,meghan kelly awful last night. getting big ego...,Negative
1,"'god blessed rep party lot great candidates, d...",Positive
2,tried watching entertainment purpose candidate...,Negative
3,look affirmative action letting black man stan...,Negative
4,seems candidate used 1st amendment right free ...,Negative
...,...,...
9651,chris wallace ask politicans finance money com...,Negative
9652,fox cherry picking candidates. jeb get softbal...,Negative
9653,i'd pay serious $$$ watch someone grill 2 egom...,Negative
9654,"""if trying destroy country here's do"" really p...",Negative


from matplotlib import pyplot as plt
import seaborn as sns
processed_data.groupby('sentiments').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [26]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
processed_data['sentiments'] = labelencoder.fit_transform(processed_data['sentiments'])

In [27]:
processed_data

,tweets,sentiments
0,meghan kelly awful last night. getting big ego...,0
1,"'god blessed rep party lot great candidates, d...",1
2,tried watching entertainment purpose candidate...,0
3,look affirmative action letting black man stan...,0
4,seems candidate used 1st amendment right free ...,0
...,...,...
9651,chris wallace ask politicans finance money com...,0
9652,fox cherry picking candidates. jeb get softbal...,0
9653,i'd pay serious $$$ watch someone grill 2 egom...,0
9654,"""if trying destroy country here's do"" really p...",0


In [28]:
# Converting Words into Vectors
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1,3))
cv.fit(processed_data['tweets'])

CountVectorizer(ngram_range=(1, 3))

In [29]:
X_train = cv.transform(processed_data['tweets'])

In [30]:
print(X_train.shape)

(9656, 81933)


In [31]:
X_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [32]:
target = processed_data['sentiments'].values

In [33]:
target

array([0, 1, 0, ..., 0, 0, 1])

In [34]:
# Sentiment Analysis (Model Building)
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()

In [35]:
classifier.fit(X_train.toarray(),target)

MultinomialNB()

In [36]:
test_tweets,test_sentiments = Clean_text(test)

In [37]:
data_test = {'tweets':test_tweets,'sentiments':test_sentiments}
final_test_data = pd.DataFrame(data_test)

In [38]:
X_test = cv.transform(final_test_data['tweets'])

In [39]:
X_test.shape

(1073, 81933)

In [40]:
y_pred = classifier.predict(X_test.toarray())

In [41]:
final_test_data['sentiments'] = labelencoder.fit_transform(final_test_data['sentiments'])

In [42]:
final_test_data

,tweets,sentiments
0,know bad trump come looking like champ situation,0
1,"hey look, it’s 10 men standing stage arguing g...",0
2,much fact checking accepted republican lie eve...,0
3,laugh mentioned pimps. get 'em mike!,1
4,gop candidate grilled. hard. expecting softbal...,0
...,...,...
1068,dr. carson talking race like trump talking money.,0
1069,"walker, paul huckabee meet putin command army ...",0
1070,"translation, jeb bush, fucked florida school b...",0
1071,nutshell via vote true blue 2016,0


In [43]:
actual_values = final_test_data['sentiments'].values

In [44]:
actual_values

array([0, 0, 0, ..., 0, 0, 0])

In [45]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred,actual_values))

0.8359739049394221


In [46]:
final_test_data

,tweets,sentiments
0,know bad trump come looking like champ situation,0
1,"hey look, it’s 10 men standing stage arguing g...",0
2,much fact checking accepted republican lie eve...,0
3,laugh mentioned pimps. get 'em mike!,1
4,gop candidate grilled. hard. expecting softbal...,0
...,...,...
1068,dr. carson talking race like trump talking money.,0
1069,"walker, paul huckabee meet putin command army ...",0
1070,"translation, jeb bush, fucked florida school b...",0
1071,nutshell via vote true blue 2016,0


In [47]:
X_test.toarray()[0]

array([0, 0, 0, ..., 0, 0, 0])

In [53]:
classifier.predict(X_test.toarray()[[0]])

array([0])